In [1]:
import os
import pandas as pd

def read_heats_file(dh_file,uncertainty,output_file):
    """
    Read the heats file written out by the MicroCal/Origin ITC analysis
    package.

    Parameters
    ----------
    dh_file : str
        name of .dh file written out by microcal software
    output_file : str
        name of file to write out data
    uncertainty : float
        user estimate of the uncertainty on each measured heat,
        treated as a percentage (e.g., 0.05 for 5%).
    Returns
    -------
    meta_data : dict
        dictionary with metadata read from the top of the file: temperature
        in Kelvin, cell and titrant concentrations in molar, and cell_volume
        in microliters
    """

    # Read data file
    with open(dh_file,'r') as f:
        lines = f.readlines()

    # Grab third line and split on ","
    meta = lines[2].split(",")

    # Parse meta data on the third line
    temperature = float(meta[0])
    stationary_cell_conc = float(meta[1])*1e-3
    titrant_syringe_conc = float(meta[2])*1e-3
    cell_volume = float(meta[3])*1e3
    
    # Split rows 6-end on "," and grab first and secon columns
    shots = []
    heats = []
    for l in lines[5:]:
        col = l.split(",")
        shots.append(float(col[0]))
        heats.append(float(col[1]))

    # Calculate heat_stdev as a percentage of each heat value
    heats_stdev = [h * uncertainty for h in heats]

    # Create the ignore_point column values
    ignore_point_column = [False for _ in range(len(heats))]
    if len(ignore_point_column) > 0:
        ignore_point_column[0] = True

    # Construct dataframe with data and write out a spreadsheet
    to_df = {"injection":shots,
             "heat":heats,
             "heat_stdev":heats_stdev,
             "ignore_point":ignore_point_column}
    df = pd.DataFrame(to_df)
    df.to_csv(output_file,index=False)

    # Build dictionary holding meta data
    out = {}
    out["temperature"] = temperature
    out["cell_conc"] = stationary_cell_conc
    out["titrant_conc"] = titrant_syringe_conc
    out["cell_volume"] = titrant_syringe_conc


    return out

In [3]:
import shutil
## Define input directory that contains .dh files
## Define output directory to put .csv files in

## Running this script twice will overwrite any previous runs of the code

inputdir = r"C:\Users\willi\Desktop\20250625"
outputdir = r"C:\Users\willi\Desktop\20250625processed" # Specify your desired output directory

def iterate_dh_to_csv(inputdir, outputdir):
    for dirpath, dirnames, filenames in os.walk(inputdir):
        for filename in filenames:
            if filename.lower().endswith('.dh'):
                filepath = os.path.join(dirpath, filename)
                
                output_filepath = os.path.splitext(filepath)[0] + ".csv"
                read_heats_file(filepath, 0, output_filepath)

                # Calculate the relative path of the .csv file within the inputdir
                rel_path = os.path.relpath(output_filepath, inputdir)

                # Create the corresponding directory structure in the outputdir
                output_dirpath = os.path.join(outputdir, os.path.dirname(rel_path))
                os.makedirs(output_dirpath, exist_ok=True)

                # Copy the .csv file to the new location
                shutil.copy2(output_filepath, output_dirpath)



iterate_dh_to_csv(inputdir, outputdir)

In [1]:
import os
import pandas as pd

def add_ignore_point_column_if_missing(folder_path):
    """
    Iterates through CSV files in the given folder and its subdirectories.
    If a CSV does not have an 'ignore_point' column, it adds one.
    In the new 'ignore_point' column, the first row will be True, 
    and subsequent rows will be False. If the CSV is empty, an empty
    'ignore_point' column is added.
    """
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                
                if 'ignore_point' not in df.columns:
                    if not df.empty:
                        # Create the new column with False for all rows initially
                        ignore_values = [False] * len(df)
                        # Set the first row to True
                        ignore_values[0] = True
                        df['ignore_point'] = ignore_values
                    else:
                        # If DataFrame is empty (no data rows), just add an empty column
                        df['ignore_point'] = [] 
                    
                    df.to_csv(filepath, index=False)

In [2]:
folder_path = r"C:\Users\willi\linkage\notebooks\data"

add_ignore_point_column_if_missing(folder_path)

In [9]:
import os
import pandas as pd

def update_heat_stdev_column(folder_path, uncertainty_percentage):
    """
    Iterates through CSV files in the given folder and its subdirectories.
    If a CSV has both 'heat' and 'heat_stdev' columns, it recalculates
    the 'heat_stdev' column using the formula: abs(heat * uncertainty_percentage).
    """
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                
                # Check if both 'heat' and 'heat_stdev' columns exist
                if 'heat' in df.columns and 'heat_stdev' in df.columns:
                    # Calculate heat_stdev and ensure it's positive
                    df['heat_stdev'] = (df['heat'] * uncertainty_percentage).abs()
                    
                    df.to_csv(filepath, index=False)

In [18]:
folder_path = r"C:\Users\willi\linkage\notebooks\data"
uncertainty_percentage = 0.001

update_heat_stdev_column(folder_path, uncertainty_percentage)

In [17]:
folder_path = r"S:\Harmslab\ITC2"
uncertainty_percentage = 0.001

update_heat_stdev_column(folder_path, uncertainty_percentage)